# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-04-14 15:09:57] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=33062, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=819305071, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=No

[2025-04-14 15:10:08 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-14 15:10:08 TP0] Init torch distributed begin.


[2025-04-14 15:10:08 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-14 15:10:08 TP0] Load weight begin. avail mem=78.58 GB


[2025-04-14 15:10:08 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-14 15:10:08 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-04-14 15:10:12 TP0] 

CUDA Graph is DISABLED.
This will cause significant performance degradation.
CUDA Graph should almost never be disabled in most usage scenarios.
If you encounter OOM issues, please try setting --mem-fraction-static to a lower value (such as 0.8 or 0.7) instead of disabling CUDA Graph.



[2025-04-14 15:10:12 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-14 15:10:13] INFO:     Started server process [3102057]
[2025-04-14 15:10:13] INFO:     Waiting for application startup.
[2025-04-14 15:10:13] INFO:     Application startup complete.
[2025-04-14 15:10:13] INFO:     Uvicorn running on http://0.0.0.0:33062 (Press CTRL+C to quit)


[2025-04-14 15:10:14] INFO:     127.0.0.1:32954 - "GET /v1/models HTTP/1.1" 200 OK
[2025-04-14 15:10:14] INFO:     127.0.0.1:32960 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-14 15:10:14 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:10:17] INFO:     127.0.0.1:32974 - "POST /generate HTTP/1.1" 200 OK
[2025-04-14 15:10:17] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-14 15:10:19 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:10:22 TP0] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 4.15, #queue-req: 0, 


[2025-04-14 15:10:22 TP0] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 99.78, #queue-req: 0, 


[2025-04-14 15:10:22 TP0] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 101.47, #queue-req: 0, 


[2025-04-14 15:10:23 TP0] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 102.41, #queue-req: 0, 


...

[2025-04-14 15:10:43 TP0] Decode batch. #running-req: 1, #token: 1892, token usage: 0.09, gen throughput (token/s): 63.28, #queue-req: 0, 


[2025-04-14 15:10:44 TP0] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 69.56, #queue-req: 0, 


[2025-04-14 15:10:44 TP0] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 102.58, #queue-req: 0, 


[2025-04-14 15:10:44 TP0] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 104.03, #queue-req: 0, 


[2025-04-14 15:10:45 TP0] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 101.29, #queue-req: 0, 
[2025-04-14 15:10:45] INFO:     127.0.0.1:51696 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-14 15:10:45 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:10:45 TP0] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 72.41, #queue-req: 0, 


[2025-04-14 15:10:46 TP0] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 84.87, #queue-req: 0, 


[2025-04-14 15:10:46 TP0] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 60.62, #queue-req: 0, 


[2025-04-14 15:10:47 TP0] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 62.95, #queue-req: 0, 


...

[2025-04-14 15:11:05 TP0] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 103.52, #queue-req: 0, 


[2025-04-14 15:11:05 TP0] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 102.98, #queue-req: 0, 


[2025-04-14 15:11:05 TP0] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 104.92, #queue-req: 0, 


[2025-04-14 15:11:06 TP0] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 101.24, #queue-req: 0, 


[2025-04-14 15:11:06] INFO:     127.0.0.1:51696 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-14 15:11:06 TP0] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:11:06 TP0] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 90.06, #queue-req: 0, 


[2025-04-14 15:11:07 TP0] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 100.84, #queue-req: 0, 


[2025-04-14 15:11:07 TP0] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 104.19, #queue-req: 0, 


[2025-04-14 15:11:07 TP0] Decode batch. #running-req: 1, #token: 159, token usage: 0.01, gen throughput (token/s): 101.62, #queue-req: 0, 


[2025-04-14 15:11:08] INFO:     127.0.0.1:51696 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-14 15:11:08 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-14 15:11:08 TP0] Decode batch. #running-req: 1, #token: 17, token usage: 0.00, gen throughput (token/s): 93.94, #queue-req: 0, 


[2025-04-14 15:11:08 TP0] Decode batch. #running-req: 1, #token: 57, token usage: 0.00, gen throughput (token/s): 106.08, #queue-req: 0, 


[2025-04-14 15:11:09 TP0] Decode batch. #running-req: 1, #token: 97, token usage: 0.00, gen throughput (token/s): 100.60, #queue-req: 0, 


[2025-04-14 15:11:09 TP0] Decode batch. #running-req: 1, #token: 137, token usage: 0.01, gen throughput (token/s): 103.96, #queue-req: 0, 


[2025-04-14 15:11:09] INFO:     127.0.0.1:51696 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-04-14 15:11:10 TP0] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-14 15:11:10 TP0] Decode batch. #running-req: 1, #token: 488, token usage: 0.02, gen throughput (token/s): 52.94, #queue-req: 0, 


[2025-04-14 15:11:10 TP0] Decode batch. #running-req: 1, #token: 528, token usage: 0.03, gen throughput (token/s): 103.25, #queue-req: 0, 


[2025-04-14 15:11:11 TP0] Decode batch. #running-req: 1, #token: 568, token usage: 0.03, gen throughput (token/s): 101.40, #queue-req: 0, 


[2025-04-14 15:11:11 TP0] Decode batch. #running-req: 1, #token: 608, token usage: 0.03, gen throughput (token/s): 102.00, #queue-req: 0, 


[2025-04-14 15:11:11 TP0] Decode batch. #running-req: 1, #token: 648, token usage: 0.03, gen throughput (token/s): 102.70, #queue-req: 0, 


...

[2025-04-14 15:11:14 TP0] Decode batch. #running-req: 1, #token: 888, token usage: 0.04, gen throughput (token/s): 101.70, #queue-req: 0, 


[2025-04-14 15:11:14 TP0] Decode batch. #running-req: 1, #token: 928, token usage: 0.05, gen throughput (token/s): 101.97, #queue-req: 0, 


[2025-04-14 15:11:14 TP0] Decode batch. #running-req: 1, #token: 968, token usage: 0.05, gen throughput (token/s): 102.21, #queue-req: 0, 


[2025-04-14 15:11:15 TP0] Decode batch. #running-req: 1, #token: 1008, token usage: 0.05, gen throughput (token/s): 97.76, #queue-req: 0, 


[2025-04-14 15:11:15 TP0] Decode batch. #running-req: 1, #token: 1048, token usage: 0.05, gen throughput (token/s): 101.46, #queue-req: 0, 
[2025-04-14 15:11:15] INFO:     127.0.0.1:51696 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = json.loads(response.json()["text"].split("</think>")[1])
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-04-14 15:11:16 TP0] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:11:16 TP0] Decode batch. #running-req: 1, #token: 57, token usage: 0.00, gen throughput (token/s): 45.31, #queue-req: 0, 


[2025-04-14 15:11:17 TP0] Decode batch. #running-req: 1, #token: 97, token usage: 0.00, gen throughput (token/s): 102.50, #queue-req: 0, 


[2025-04-14 15:11:17 TP0] Decode batch. #running-req: 1, #token: 137, token usage: 0.01, gen throughput (token/s): 107.01, #queue-req: 0, 


[2025-04-14 15:11:17 TP0] Decode batch. #running-req: 1, #token: 177, token usage: 0.01, gen throughput (token/s): 101.23, #queue-req: 0, 


...

[2025-04-14 15:11:21 TP0] Decode batch. #running-req: 1, #token: 577, token usage: 0.03, gen throughput (token/s): 93.85, #queue-req: 0, 


[2025-04-14 15:11:22 TP0] Decode batch. #running-req: 1, #token: 617, token usage: 0.03, gen throughput (token/s): 100.63, #queue-req: 0, 


[2025-04-14 15:11:22 TP0] Decode batch. #running-req: 1, #token: 657, token usage: 0.03, gen throughput (token/s): 102.28, #queue-req: 0, 


[2025-04-14 15:11:22 TP0] Decode batch. #running-req: 1, #token: 697, token usage: 0.03, gen throughput (token/s): 102.98, #queue-req: 0, 


[2025-04-14 15:11:23] INFO:     127.0.0.1:51116 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not exactly sure where the capital of France is, but I think it\'s Paris. Yeah, I remember hearing that Paris is the capital. Let me think about what details I should include. \n\nFirst, the basic info: country, city, population, and maybe some key landmarks. I know the population is around 2 million, but I\'m not sure of the exact number. I think it\'s approximately 2,165,000. As for landmarks, the Eiffel Tower is a must. The Louvre Museum is another famous spot. The Arc de Triomphe is also iconic. Maybe the Seine River is important too since it\'s a major river in the city.\n\nI should structure this in JSON. So, the main object would have a "country" key pointing to "France," and the "capital" key pointing to "Paris." Under "capital," I can have an "info" array that includes population, landmarks

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-04-14 15:11:23 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-14 15:11:23 TP0] Decode batch. #running-req: 1, #token: 11, token usage: 0.00, gen throughput (token/s): 99.30, #queue-req: 0, 


[2025-04-14 15:11:23 TP0] Decode batch. #running-req: 1, #token: 51, token usage: 0.00, gen throughput (token/s): 102.04, #queue-req: 0, 


[2025-04-14 15:11:24 TP0] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, gen throughput (token/s): 104.18, #queue-req: 0, 


[2025-04-14 15:11:24 TP0] Decode batch. #running-req: 1, #token: 131, token usage: 0.01, gen throughput (token/s): 106.93, #queue-req: 0, 


[2025-04-14 15:11:24 TP0] Decode batch. #running-req: 1, #token: 171, token usage: 0.01, gen throughput (token/s): 100.89, #queue-req: 0, 


...

[2025-04-14 15:11:42 TP0] Decode batch. #running-req: 1, #token: 1891, token usage: 0.09, gen throughput (token/s): 101.00, #queue-req: 0, 


[2025-04-14 15:11:43 TP0] Decode batch. #running-req: 1, #token: 1931, token usage: 0.09, gen throughput (token/s): 103.58, #queue-req: 0, 


[2025-04-14 15:11:43 TP0] Decode batch. #running-req: 1, #token: 1971, token usage: 0.10, gen throughput (token/s): 106.15, #queue-req: 0, 


[2025-04-14 15:11:44 TP0] Decode batch. #running-req: 1, #token: 2011, token usage: 0.10, gen throughput (token/s): 103.32, #queue-req: 0, 


[2025-04-14 15:11:44 TP0] Decode batch. #running-req: 1, #token: 2051, token usage: 0.10, gen throughput (token/s): 103.52, #queue-req: 0, 
[2025-04-14 15:11:44] INFO:     127.0.0.1:60236 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-04-14 15:11:44 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-14 15:11:44 TP0] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:11:45 TP0] Decode batch. #running-req: 3, #token: 122, token usage: 0.01, gen throughput (token/s): 172.19, #queue-req: 0, 


[2025-04-14 15:11:45 TP0] Decode batch. #running-req: 3, #token: 242, token usage: 0.01, gen throughput (token/s): 281.88, #queue-req: 0, 


[2025-04-14 15:11:45 TP0] Decode batch. #running-req: 3, #token: 362, token usage: 0.02, gen throughput (token/s): 282.10, #queue-req: 0, 


[2025-04-14 15:11:46 TP0] Decode batch. #running-req: 3, #token: 482, token usage: 0.02, gen throughput (token/s): 299.09, #queue-req: 0, 


...

[2025-04-14 15:12:04 TP0] Decode batch. #running-req: 3, #token: 5642, token usage: 0.28, gen throughput (token/s): 291.00, #queue-req: 0, 


[2025-04-14 15:12:04 TP0] Decode batch. #running-req: 3, #token: 5762, token usage: 0.28, gen throughput (token/s): 280.32, #queue-req: 0, 


[2025-04-14 15:12:05 TP0] Decode batch. #running-req: 3, #token: 5882, token usage: 0.29, gen throughput (token/s): 282.32, #queue-req: 0, 


[2025-04-14 15:12:05 TP0] Decode batch. #running-req: 3, #token: 6002, token usage: 0.29, gen throughput (token/s): 289.71, #queue-req: 0, 


[2025-04-14 15:12:05 TP0] Decode batch. #running-req: 3, #token: 6122, token usage: 0.30, gen throughput (token/s): 297.28, #queue-req: 0, 
[2025-04-14 15:12:06] INFO:     127.0.0.1:43588 - "POST /generate HTTP/1.1" 200 OK
[{'text': "600 words.\n\nThe capital of France is Paris. Paris is one of the most important cities in the world, and it's also the political, cultural, and economic center of France. The city has a rich history that dates back to ancient times, and it's known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe. Paris is also famous for its cuisine, with dishes like baguette, croissant, and boeuf bourguignon being some of the most popular. The city is surrounded by the Seine River, which flows through it, and the bridges over the river add to the city's charm. Paris is a vibrant city with a mix of old-world charm and modern innovation, making it a unique and fascinating place to visit.\n\nThe capital of France is Paris. Paris 

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-04-14 15:12:06 TP0] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:12:06 TP0] Decode batch. #running-req: 1, #token: 35, token usage: 0.00, gen throughput (token/s): 140.73, #queue-req: 0, 


[2025-04-14 15:12:06 TP0] Decode batch. #running-req: 1, #token: 75, token usage: 0.00, gen throughput (token/s): 104.77, #queue-req: 0, 


[2025-04-14 15:12:07 TP0] Decode batch. #running-req: 1, #token: 115, token usage: 0.01, gen throughput (token/s): 104.87, #queue-req: 0, 


[2025-04-14 15:12:07 TP0] Decode batch. #running-req: 1, #token: 155, token usage: 0.01, gen throughput (token/s): 106.70, #queue-req: 0, 


...

[2025-04-14 15:12:24 TP0] Decode batch. #running-req: 1, #token: 1875, token usage: 0.09, gen throughput (token/s): 103.97, #queue-req: 0, 


[2025-04-14 15:12:24 TP0] Decode batch. #running-req: 1, #token: 1915, token usage: 0.09, gen throughput (token/s): 101.41, #queue-req: 0, 


[2025-04-14 15:12:24 TP0] Decode batch. #running-req: 1, #token: 1955, token usage: 0.10, gen throughput (token/s): 106.18, #queue-req: 0, 


[2025-04-14 15:12:25 TP0] Decode batch. #running-req: 1, #token: 1995, token usage: 0.10, gen throughput (token/s): 103.16, #queue-req: 0, 


[2025-04-14 15:12:25 TP0] Decode batch. #running-req: 1, #token: 2035, token usage: 0.10, gen throughput (token/s): 102.99, #queue-req: 0, 
[2025-04-14 15:12:25] INFO:     127.0.0.1:53286 - "POST /generate HTTP/1.1" 200 OK
{'text': ' the \\( n \\9121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121212121

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-04-14 15:12:25 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-14 15:12:26 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 96.25, #queue-req: 0, 


[2025-04-14 15:12:26 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 103.08, #queue-req: 0, 


[2025-04-14 15:12:26 TP0] Decode batch. #running-req: 1, #token: 121, token usage: 0.01, gen throughput (token/s): 102.99, #queue-req: 0, 


[2025-04-14 15:12:27 TP0] Decode batch. #running-req: 1, #token: 161, token usage: 0.01, gen throughput (token/s): 105.70, #queue-req: 0, 


...

[2025-04-14 15:12:29 TP0] Decode batch. #running-req: 1, #token: 361, token usage: 0.02, gen throughput (token/s): 106.00, #queue-req: 0, 


[2025-04-14 15:12:29 TP0] Decode batch. #running-req: 1, #token: 401, token usage: 0.02, gen throughput (token/s): 101.19, #queue-req: 0, 


[2025-04-14 15:12:30 TP0] Decode batch. #running-req: 1, #token: 441, token usage: 0.02, gen throughput (token/s): 105.40, #queue-req: 0, 


[2025-04-14 15:12:30 TP0] Decode batch. #running-req: 1, #token: 481, token usage: 0.02, gen throughput (token/s): 103.22, #queue-req: 0, 


[2025-04-14 15:12:30] INFO:     127.0.0.1:43460 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-04-14 15:12:30] Child process unexpectedly failed with an exit code 9. pid=3102395
[2025-04-14 15:12:30] Child process unexpectedly failed with an exit code 9. pid=3102323


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.27s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.20s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.21s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text:  and also, make sure that the JSON is valid.

```json
{
  "name": "Beijing",
  "population": 10000000,
  "area": 100000,
  "founded": 1500,
  "coordinates": {
    "latitude": "40.4168",
    "longitude": "-73.9352"
  }
}
```

Is this JSON valid? If not, explain why.

If it is valid, explain why.

Also, provide an updated version of the JSON with any necessary corrections.
</think>{

"name": "Beijing",
"population": 1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text:  and also, make sure that the JSON is valid.

```json
{
  "name": "Beijing",
  "population": 10000000,
  "area": 100000,
  "founded": 1500,
  "coordinates": {
    "latitude": "40.4168",
    "longitude": "-73.9352"
  }
}
```

Is this JSON valid? If not, explain why.

If it is valid, explain why.

Also, provide an updated version of the JSON with any necessary corrections.
</think>{

"name": "Beijing",
"population": 1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 9-10 sentences long.

The capital of France is Paris. It is a historic city and has been the capital since ancient times. Paris is known for its rich history, stunning landmarks like the Eiffel Tower, and cultural attractions such as the Louvre Museum and the Paris Opera House. The city is located in the northern part of France, bordered by the Seine River, which flows through it, and the River Meuse on the east. Paris is an important economic and political center of France and has a significant influence in European and global affairs. The city is also known for its vibrant culture, with numerous museums, theaters
Prompt: Give me the information of the capital of Germany.
Generated text: 85

<think>
Okay, I need to provide information about the capital of Germany, which is Berlin. Let me start by recalling what I know about Berlin. I know it's the seat of the government, but I'm not exactly sure about all the de

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its history, culture, economy, and society..
London is the capital of the United Kingdom and has long been considered one of the world's most significant cities. It is renowned for its rich history, diverse culture, and vibrant economy. The city has a long history dating back to ancient times, with contributions from various civilizations such as the Romans, Anglo-Saxons, and Normans. London's history is marked by iconic landmarks like the Tower of London, Big Ben, and the Houses of Parliament. The city's culture is multicultural, with a blend of English, European, African, Asian, and Middle Eastern influences. London's
Prompt: Please provide information about Paris as a major global city:
Generated text:  1) location and elevation; 2) population; 3) economic activities; 4) landmarks; 5) transportation; 6) cultural aspects; 7) environmental issues; 8) historical significance; and 9) education and c

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Okay, so I need to create a JSON object about the historical capital of France, which is Paris. Hmm, I'm a bit new to this, but I think I can figure it out. Let me start by recalling what Paris is known for. It's the capital, right? I remember it's a major city in France and has a rich history.

First, I guess I should include some basic information. Maybe the population? I think Paris is one of the largest cities in France, but I'm not sure of the exact figure. I'll have to check that. Oh, wait, perhaps it's around 3 million? That sounds familiar.

Next, I should include the area. Paris is probably quite large, but I'm not certain. I think it's a bit over 100 square kilometers. Yeah, that seems right. Now, landmarks are important too. The Eiffel Tower is a must. I know it's a symbol. Also, the Louvre Museum, the RATP, which is the public trans

In [19]:
llm.shutdown()